## LDA 속성분석

참고한 사이트: https://soyoung-new-challenge.tistory.com/46

In [4]:
import re
import os
import sys
import json

from konlpy.tag import Kkma
from konlpy.tag import Okt
import soynlp

import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import *
from konlpy.utils import pprint
import numpy as np
import lda
from sklearn.pipeline import Pipeline
import pandas as pd
import statsmodels.api as sm
from collections import Counter

In [1]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-xs7jzjpk
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-xs7jzjpk
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [6]:
from pykospacing import Spacing      

## 리뷰 데이터(우리가 작업한 데이터 가져오기)

In [7]:
kkma = Kkma()
okt = Okt()

In [8]:
import os
from os.path import join
import pandas as pd

data_dir = os.getenv('HOME')+'/Desktop/project/data'

df = join(data_dir, 'google_reviews.csv')
df = pd.read_csv(df)
df.head()

,name,ratings,date,label,comment,search,keyword
0,미샤,4.0,2주 전,1.0,여기는 갈 때마다 새로운 느낌이 야 즐거운 시간이 되었습니다,장태산자연휴양림메타세콰이어산림욕장,jangtaesan
1,Gyang gree young,5.0,3주 전,1.0,하늘 보고 누워 힐링하기 너무 좋은 곳이에요,장태산자연휴양림메타세콰이어산림욕장,jangtaesan
2,박진수,5.0,3달 전,1.0,잘 정리된 메타세쿼이아 숲에서 산림욕하기 좋은 곳으로 둘레길 걷기와 전망대까지 가벼...,장태산자연휴양림메타세콰이어산림욕장,jangtaesan
3,Sh Choi,5.0,7달 전,1.0,산책하기 너무 좋은 명소,장태산자연휴양림메타세콰이어산림욕장,jangtaesan
4,박은선,5.0,1년 전,1.0,주말 마자 애정 하는 장태산 나들이 단풍도 예뻐요,장태산자연휴양림메타세콰이어산림욕장,jangtaesan


In [9]:
df.tail(50)

,name,ratings,date,label,comment,search,keyword
8358,김경화,4.0,1년 전,1.0,어린아이들이 놀기 편해 요 상록 놀이동산보다 조금 크고 서울랜드보다 작고 그래요 기...,오월드,o_world
8359,jian k,5.0,1년 전,0.0,음,오월드,o_world
8360,막대사탕,1.0,1년 전,0.0,같이 할 게 없고 바이킹도 재미없다,오월드,o_world
8361,21 8,4.0,1년 전,1.0,버드랜드 새 모이주 기 너무 재밌어 오,오월드,o_world
8362,Richer Jak,5.0,1년 전,1.0,가격 대비 정말 볼거리 많고 가족끼리 오기 좋은 장소입니다 멀리서 오시는 분들은 주...,오월드,o_world
8363,김광길,5.0,1년 전,1.0,볼 게 많다 아이들이 뛰어놀 수 있어요,오월드,o_world
8364,Sung-Do Oh,4.0,1년 전,1.0,좋아요,오월드,o_world
8365,Thinus Scholtz,4.0,1년 전,1.0,놀라운 곳이지만 여름이나 봄에 갈 것을 안 합니다 지금까지 한국에서 본 최고의 동물,오월드,o_world
8366,이불leebul,5.0,1년 전,1.0,진짜 재밌고 사파리도 있어서 재밌어 여 썰매장도 있고요,오월드,o_world
8367,Asraf Masum,4.0,1년 전,1.0,환상적인 장소,오월드,o_world


In [10]:
df.isnull().sum()              # 키워드 2190개 없음

name         30
ratings      30
date         30
label        30
comment      30
search       30
keyword    2190
dtype: int64

In [11]:
df = df.drop(df.index[8378:8408])        # 값이 없는 행 삭제

In [12]:
df.tail()

,name,ratings,date,label,comment,search,keyword
8373,young wu Kwon,5.0,1년 전,1.0,재밌어요,오월드,o_world
8374,강강수월래,4.0,1년 전,1.0,애들 놀고 구경거리는 굿,오월드,o_world
8375,원언니,5.0,1년 전,1.0,초등생 전용 놀이로 아담하고 이 뻐요,오월드,o_world
8376,김경영,4.0,1년 전,1.0,괜찮아요,오월드,o_world
8377,김소연,5.0,1년 전,1.0,겨울에도 놀기 좋아요,오월드,o_world


In [13]:
df = df.drop(['name','ratings', 'date', 'label', 'keyword'], axis=1)      # 불필요한 열 제거

In [14]:
df

,comment,search
0,여기는 갈 때마다 새로운 느낌이 야 즐거운 시간이 되었습니다,장태산자연휴양림메타세콰이어산림욕장
1,하늘 보고 누워 힐링하기 너무 좋은 곳이에요,장태산자연휴양림메타세콰이어산림욕장
2,잘 정리된 메타세쿼이아 숲에서 산림욕하기 좋은 곳으로 둘레길 걷기와 전망대까지 가벼...,장태산자연휴양림메타세콰이어산림욕장
3,산책하기 너무 좋은 명소,장태산자연휴양림메타세콰이어산림욕장
4,주말 마자 애정 하는 장태산 나들이 단풍도 예뻐요,장태산자연휴양림메타세콰이어산림욕장
...,...,...
8373,재밌어요,오월드
8374,애들 놀고 구경거리는 굿,오월드
8375,초등생 전용 놀이로 아담하고 이 뻐요,오월드
8376,괜찮아요,오월드


In [ ]:
# df2 = df.set_index("search")        # 인덱스 변경

In [15]:
df2 = df.to_dict("list")                # 딕셔너리 변경(딕셔너리의 값이 리스트형으로 반환된다)              

In [16]:
comment_data = df2       # 그대로 함수를 사용하기 위해서 명 변경

## 파일을 딕셔너리 형태로 변환

In [17]:
# 파일을 사용하기 좋게 딕셔너리 형태로 변환(가져와서 쓰는 함수 그대로 쓰기 위해 일단 함수명 그대로 사용)

restaurant_review = []      

for key in comment_data.keys():          
    temp_dict={}          # 레스토랑 이름(search)을  id에 담고, 그에 해당하는 블로그 리뷰(comment)는 review에 담는다

    temp_dict['_id'] = key.split(' ')[0]   #공백으로 키를 나누고 첫번째 것을 _id에 저장 
    temp_dict['review'] = comment_data[key]   #코멘트 데이터의 키를 리뷰로 저장
    if temp_dict['review'] == []:
        continue
    restaurant_review.append(temp_dict)

In [24]:
restaurant_review

[{'_id': 'comment',
  'review': ['여기는 갈 때마다 새로운 느낌이 야 즐거운 시간이 되었습니다',
   '하늘 보고 누워 힐링하기 너무 좋은 곳이에요',
   '잘 정리된 메타세쿼이아 숲에서 산림욕하기 좋은 곳으로 둘레길 걷기와 전망대까지 가벼운 등산 활동까지 할 수 있습니다 숲속의 집을 이용하거나 스카이워크 시설과 출렁다리를 통해 다양한 체험을 즐길 수 있습니다',
   '산책하기 너무 좋은 명소',
   '주말 마자 애정 하는 장태산 나들이 단풍도 예뻐요',
   '무료인데다가 쉴 곳이 많고 시설이 잘 되어 있다',
   '관람료 무료여서 좋고요 전망대까지 분 정도 걸리는 완만한 산책길은 대화하면서 올라가기 좋습니다 책 한 권 갖고 가서 그늘에 앉아 읽고 오기 정말 좋은 곳입니다로 잼의 도시 우리 대전에는 산 산 산장 태산이 있습니다',
   '대전의 좋은 산림욕장',
   '힐링 되고 아주 좋네요',
   '힐링 코스',
   '주위기가 너무 새로워요',
   '메타세콰이 아이숲이 울창하고 좋네요 산책하기에도 좋고 담에는 맛있는 음식 가져와 돗자리에서 먹고 누워서 하늘도 보고 싶네요 숲이 주는 향기가 좋아요',
   '좋아요',
   '좋아요',
   '장태산은 메타세쿼이아 나무가 높이높이 솟아 있어서 장관입니다 전망대나 흔들 다리도 잘 만들어져 있어서 구경도 잘하고 산림욕하기 좋았습니다',
   '기 좋아요',
   '좋아요',
   '메타 크로스 나무가 너무 멋있고 기 좋고 푸르름에 건강해지는 느낌 진짜 좋았어요',
   '다시 한가 보고 싶은 곳입니다',
   '정말 좋아요',
   '저 주일에 오랜만에 텐트를 펼치고 잠시나마 아이들과 휴식을 하고 왔습니다',
   '가볍게 산책하기 좋은 곳입니다',
   '쥐라기부터 단일 품종으로 생존하고 있는 메타세쿼이아 숲이 주연하는 초록의 향연을 맛보고 싶다면 꼭 일부러 시간 내 들러보셔요',
   '시힐링 최고',
   '기가 너무 좋아요 가족과 함께하는 시간을 즐겨보세요',
  

## 함수

In [18]:
# 전처리 함수

def preprocessing(review):
    total_review = ''
    #인풋리뷰
    for idx in range(len(review)):
        r = review[idx]
        #하나의 리뷰에서 문장 단위로 자르기
        for sentence in kkma.sentences(r):
            sentence = re.sub('([a-zA-Z])','',sentence)
            sentence = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',sentence)
            sentence = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',sentence)
            if len(sentence) == 0:
                continue
            if len(sentence) < 198:
                sentence = spacing(sentence)     # 띄어쓰기
            sentence += '. '
            total_review += sentence
    return total_review

In [19]:
# 감성사전 생성(추후에 변경)

service_good_feature = {'서비스':['좋','친절','괜찮','최고','빠르','짱','훌륭','추천','감사','구수','최상','대박',
                               '훈훈','특별','개이득','최고','만족','세련','최고','감동'],
                        '사장':['친절','스윗','센스'],
                        '알바':['친절','스윗','센스'],
                        '직원': ['친절','스윗','센스'],
                        '일을':['잘','빠르게'],
                        '일도':['잘','빠르게'],
                        '서빙':['잘','빠르게']}

service_bad_feature = {'서비스': ['아쉽','최악','나쁘','느리','빡치','비추','별로','그냥','낙제','쏘다쏘다','엉망','실망','불친절','문제','컴플레인',
                               '거지','그닥','그다지','구려','불편','엉성','헬','개판'],
                       '알바':['불친절','똑바로','재수'],
                       '사장':['불친절','똑바로','재수'],
                       '직원':['불친절','똑바로','재수'],
                      '일을':['못','느리게','답답'],
                      '일도':['못','느리게', '천천히'],
                      '서빙':['못','느리게','천천히','답답']}

taste_good_feature = {'간':['맞','적절','딱','환상','담백'],
                      '음식':['깔끔'],
                      '맛':['있','좋','나다','최고']}

taste_bad_feature = {'간':['세','쎄','강하다','별로'],
                     '음식':['별로','쏘다쏘다','최악'],
                     '맛':['별로','최악']}

taste_good_emotion = ['고소','부드럽','신선','촉촉','싱싱','정갈','최고']

taste_bad_emotion = ['싱겁','느끼다하다','짜다','느끼다','짜다','딱딱하다','차갑다']

cost_good_feature = {'가격': ['괜찮','착하다','저렴','적당','싸다','좋다','합리적','훌륭','최고','만족','마음','든든','알맞다',
                            '무난','괜춘','최상','최상','굿','엄지','낮'],
                     '가성비':['괜찮','착하다','저렴','적당','싸다','좋다','합리적','훌륭','최고','만족','마음','든든','알맞다',
                            '무난','괜춘','최상','최상','굿','엄지'],
                     '양':['많','적당','푸짐하고','괜찮다','넉넉','충분','든든']
                     }

cost_bad_feature ={'가격': ['비싸','있다','나쁘','사악','비효율','높다','부담','아쉽','쏘다쏘다','별로','그닥','그다지','쎄','ㅎㄷㄷ','높','거품'],
                   '가성비':['별로'],
                   '양':['적다','작다','아쉽','적다','다소','별로'],
                   }

atmosphere_good_feature = {'분위기': ['좋','괜찮','조용','깔끔','적당','깡패','굉장','아담','완벽','아기자기','고급','최고','세련','만족','아늑','훌륭','예쁘','이쁘','짱',
                                   '심쿵','따뜻','깨끗','독특','매력','모던','취향저격','맘','마음','클래식','아름','인상적','귀엽','포근'], 
                           '인테리어': ['좋','괜찮','조용','깔끔','적당','깡패','굉장','아담','완벽','아기자기','고급','최고','세련','만족','아늑','훌륭','예쁘','이쁘','짱',
                                    '심쿵','따뜻','깨끗','독특','매력','모던','취향저격','맘','마음','클래식','아름','인상적','귀엽','포근']}

atmosphere_bad_feature = {'분위기': ['나쁘다','바쁘다','어수선하다','이상하다','촌스럽다','별로','부담스럽다','시끄럽','복잡' ],
                          '인테리어':[]}

visit_good_feature = {'의사':['있다','충만','백프로','백프롭','많','만땅','마구','그득','만점','넘침'],
                      '다시':['가다','오다','방문','찾다','가보다','한번','갈다','찾아가다','가야지','갈거다','방문하다보고',
                            '생각나다','방문한다면','와보고','재방문','접하다','간다면','갈다때가','먹다고프다','방문한다임','오자고','가기로','갈다생각이다','가면'],
                      '굳이':[]}

visit_bad_feature = {'의사':['글쎄'],
                     '굳이':['다시','많이','여기까지','줄서서','찾아','시키다','가다','찾다','여기','기다리다','줄을','사먹'],
                     '다시':[]}

negative_word_emotion = ['안','않','못','없','아닌','아니']

In [20]:
# 특정 키워드 추출하는 함수

def get_feature_keywords(feature_keywords, review):
    feature_temp = []
    for keyword in feature_keywords:
        if re.findall(keyword, review):
            sub_list = ['게','고','음','며','데','만','도','면']
            
            for sub in sub_list:
                if sub+' ' in review:
                    review = re.sub(sub+' ', sub+',', review)
                
            a = re.findall(keyword +'+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review) # K한 한 한글
            b = re.findall(keyword + '+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review) # K 한 한글 
            c = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+' + keyword +'[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review) # 한 K한글 예쁜 분위기가
                
            for ngram in a:
                t = ()
                feature_temp.append(t + (ngram,keyword))
            for ngram in b:
                t = ()
                feature_temp.append(t + (ngram,keyword))
            for ngram in c:
                t = ()
                feature_temp.append(t + (ngram,keyword))
                
    return feature_temp

In [21]:
# 특정 키워드 감정을 찾는 함수

def get_feature_emotions(feature_good_dict,feature_bad_dict, feature_temp):
    good_feature_emotion_list = []
    bad_feature_emotion_list = []
    
    for ngrams in feature_temp:
        keyword = ngrams[1]
        ngram = ngrams[0]
        is_bad_feature = None
        
        good_emotion_list = feature_good_dict[keyword]
        bad_emotion_list = feature_bad_dict[keyword]
        for emotion in good_emotion_list:
            if re.findall(emotion, ngram):
                is_bad_feature = False  
        for emotion in bad_emotion_list:
            if re.findall(emotion, ngram):
                is_bad_feature = True    
        for negative in negative_word_emotion:
            if re.findall(negative, ngram):
                if is_bad_feature == True:
                    is_bad_feature = False
                    break
                elif is_bad_feature == False:
                    is_bad_feature = True
                    break
                else:
                    is_bad_feature = True
                    break   
        if is_bad_feature:
            bad_feature_emotion_list.append(ngram)
        elif is_bad_feature == False:
            good_feature_emotion_list.append(ngram)
        else:
            pass
    return good_feature_emotion_list, bad_feature_emotion_list

In [22]:
# 맛에 대한 감정을 찾는 함수

def get_taste_emotion(taste_good_emotions, taste_bad_emotions):
    bad_taste_emotion_list = []
    good_taste_emotion_list = []
    for ngrams in taste_good_emotions:
        ngram = ngrams[0]
        is_bad_taste = False
        for negative in negative_word_emotion:
            if re.findall(negative,ngram):
                is_bad_taste = True
        if is_bad_taste:
            bad_taste_emotion_list.append(ngram)
        else:
            good_taste_emotion_list.append(ngram)
    
    for ngrams in taste_bad_emotions:
        ngram = ngrams[0]
        is_bad_taste = True
        for negative in negative_word_emotion:
            if re.findall(negative,ngram):
                is_bad_taste = False
        if is_bad_taste:
            bad_taste_emotion_list.append(ngram)
        else:
            good_taste_emotion_list.append(ngram)
            
    return good_taste_emotion_list, bad_taste_emotion_list

In [23]:
# 최종 테스트 함수

check_division = lambda x, y: y if y ==0 else round((x / float(y)),2)

for i, restaurant in enumerate(restaurant_review):
    restaurant_good_service_count = 0
    restaurant_bad_service_count = 0
    restaurant_good_atmosphere_count =0
    restaurant_bad_atmosphere_count =0
    restaurant_good_cost_count =0
    restaurant_bad_cost_count =0
    restaurant_good_visit_count = 0
    restaurant_bad_visit_count = 0
    restaurant_good_taste_count = 0
    restaurant_bad_taste_count = 0
    print(i, restaurant['_id'])
    reviews_list = restaurant
    for j, review in enumerate(reviews_list):
        service_temp = get_feature_keywords(service_good_feature.keys(), review)
        good_service,bad_service = get_feature_emotions(service_good_feature, service_bad_feature, service_temp)

        atmosphere_temp = get_feature_keywords(atmosphere_good_feature.keys(), review)
        good_atmosphere,bad_atmosphere = get_feature_emotions(atmosphere_good_feature, atmosphere_bad_feature, atmosphere_temp)

        cost_temp = get_feature_keywords(cost_good_feature.keys(), review)
        good_cost,bad_cost = get_feature_emotions(cost_good_feature, cost_bad_feature, cost_temp)

        visit_temp = get_feature_keywords(visit_good_feature.keys(), review)
        good_visit,bad_visit = get_feature_emotions(visit_good_feature, visit_bad_feature, visit_temp)

        taste_temp = get_feature_keywords(taste_good_feature.keys(), review)
        good_taste,bad_taste = get_feature_emotions(taste_good_feature, taste_bad_feature, taste_temp)
        taste_good_emotion_temp = get_feature_keywords(taste_good_emotion, review)
        taste_bad_emotion_temp = get_feature_keywords(taste_bad_emotion, review)
        good_taste2, bad_taste2 = get_taste_emotion(taste_good_emotion_temp,taste_bad_emotion_temp)
        good_taste.extend(good_taste2)
        bad_taste.extend(bad_taste2)

        if len(good_service) > len(bad_service):
            restaurant_good_service_count += 1
        elif len(good_service) < len(bad_service):
            restaurant_bad_service_count += 1
        else:
            pass
        
        if len(good_atmosphere) > len(bad_atmosphere):
            restaurant_good_atmosphere_count += 1
        elif len(good_atmosphere) < len(bad_atmosphere):
            restaurant_bad_atmosphere_count += 1
        else:
            pass
        
        if len(good_cost) > len(bad_cost):
            restaurant_good_cost_count += 1
        elif len(good_cost) < len(bad_cost):
            restaurant_bad_cost_count += 1
        else:
            pass
            
        if len(good_visit) > len(bad_visit):
            restaurant_good_visit_count += 1
        elif len(good_visit) < len(bad_visit):
            restaurant_bad_visit_count += 1
        else:
            pass
        
        if len(good_taste) > len(bad_taste):
            restaurant_good_taste_count += 1
        elif len(good_taste) < len(bad_taste):
            restaurant_bad_taste_count += 1
        else:
            pass
        
    TT = restaurant_good_service_count + restaurant_bad_service_count + restaurant_good_taste_count + restaurant_bad_taste_count + restaurant_good_atmosphere_count + restaurant_bad_atmosphere_count + restaurant_good_cost_count + restaurant_bad_cost_count
    
    if TT > 5:
        print('Total review count: {}'.format(len(reviews_list)))
        print('Good service: {}/{} = {}'.format(restaurant_good_service_count,restaurant_good_service_count + restaurant_bad_service_count,100*check_division(restaurant_good_service_count, restaurant_good_service_count + restaurant_bad_service_count)))
        print('Good atmosphere: {}/{} = {}'.format(restaurant_good_atmosphere_count,restaurant_good_atmosphere_count + restaurant_bad_atmosphere_count,100*check_division(restaurant_good_atmosphere_count,restaurant_good_atmosphere_count + restaurant_bad_atmosphere_count))) 
        print('Good cost: {}/{} = {}'.format(restaurant_good_cost_count,restaurant_good_cost_count + restaurant_bad_cost_count, 100*check_division(restaurant_good_cost_count,restaurant_good_cost_count + restaurant_bad_cost_count)))
        print('Good taste: {}/{} = {}'.format(restaurant_good_taste_count,restaurant_good_taste_count + restaurant_bad_taste_count,100*check_division(restaurant_good_taste_count,restaurant_good_taste_count + restaurant_bad_taste_count)))
        print('')

0 comment
1 search
